# Plota mapa diário de precipitação do MERGE
 - Dados MERGE fornecido pelo CPTEC: https://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/
 - Realizado por: Enrique V. Mattos - 28/08/2025

# **1° Passo:** Preparando ambiente

In [ ]:
# instalações
!pip install -q ultraplot cartopy pygrib rioxarray geobr salem folium

# monta o drive
from google.colab import drive
drive.mount('/content/drive')

# nome do diretório
dir = '/content/drive/MyDrive/2-PESQUISA/0_GLM/estudos_de_caso/2025-08-22-FRENTEFRIA_RS'

# importa bibliotecas
import ultraplot as uplt
import xarray as xr
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import os
import glob
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import time
from matplotlib import cm
import matplotlib.patches as patches
import pygrib
import matplotlib
import rioxarray as rxr
from datetime import datetime, timedelta
import pandas as pd
import salem
import geobr

import warnings
warnings.filterwarnings("ignore")

# **2° Passo:** Declarando funções

In [ ]:
def evm_plot_states(shapefile, cor, espessura_linha):
    import cartopy.crs as ccrs
    import cartopy.io.shapereader as shpreader
    import matplotlib.pyplot as plt
    shapefile = list(shpreader.Reader(shapefile).geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor=cor, facecolor='none', linewidth=espessura_linha)

# shapefile: Município
def carrega_shape_municipio(municipio):

    municipios_geobr = geobr.read_municipality(code_muni="all", year=2020)

    municipio_geobr = municipios_geobr [municipios_geobr['name_muni'] == municipio]

    if municipios_geobr.empty:
        raise ValueError(f"Não encontrado o estado '{estado}'.")
    municipio_geobr.crs = 'epsg:4674'

    return municipio_geobr

# **Processando os dados do MERGE**

## Baixando os dados

In [ ]:
%%time
# ano, mes e dia INICIAL do período
anoi = 2025
mesi = 8
diai = 20

# ano, mes e dia FINAL do período
anof = 2025
mesf = 8
diaf = 24

# FTP dos dados por dia
ftp = 'http://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/'

# Data desejada
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

# Número de dias
n_days = date_en - date_in

# Download dos dados
for file in pd.date_range(date_ini, date_end, freq='d'):

    # extrai ano e mes
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')

    # nome do arquivo
    filename = f'MERGE_CPTEC_{ano}{mes}{dia}.grib2'

    # ftp + filename. Exemplo: http://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/2023/09/MERGE_CPTEC_20230920.grib2
    ftp_filename = f'{ftp}{ano}/{mes.zfill(2)}/{filename}'

    # baixando arquivo diário de chuva
    !wget {ftp_filename}

    print('\n---------------------')
    print('Downloading FTP File:')
    print('---------------------')
    print('Model: MERGE')
    print('File Name: ' + ftp_filename)

## Processando os dados

In [ ]:
%%time
#==================================================================================================#
#                              DEFINIÇÃO DE PARÂMETROS
#==================================================================================================#
# limites das latitudes e longitudes
lonmin, lonmax, latmin, latmax = -60.0, -40., -40., -20.0

# extensão da imagem [min. lon, min. lat, max. lon, max. lat]
extent = [lonmin, latmin, lonmax, latmax]

#==================================================================================================#
#                                     SHAPEFILES
#==================================================================================================#
shp_ES = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/ES/ES_UF_2019.shp').geometries())
shp_RJ = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/RJ/RJ_UF_2019.shp').geometries())
shp_SP = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/SP/SP_UF_2019.shp').geometries())
shp_PR = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/PR/PR_UF_2019.shp').geometries())
shp_SC = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/SC/SC_UF_2019.shp').geometries())
shp_RS = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/RS/RS_UF_2019.shp').geometries())

#==================================================================================================#
#                                     LEITURA MERGE (GRIB)
#==================================================================================================#
# DIA 20
grib = pygrib.open('/content/MERGE_CPTEC_20250820.grib2')
grb = grib.select(name='Precipitation')[0]
precip_dia20, lats, lons = grb.data(lat1=extent[1],lat2=extent[3],lon1=extent[0]+360,lon2=extent[2]+360)

# DIA 21
grib = pygrib.open('/content/MERGE_CPTEC_20250821.grib2')
grb = grib.select(name='Precipitation')[0]
precip_dia21, lats, lons = grb.data(lat1=extent[1],lat2=extent[3],lon1=extent[0]+360,lon2=extent[2]+360)

# DIA 22
grib = pygrib.open('/content/MERGE_CPTEC_20250822.grib2')
grb = grib.select(name='Precipitation')[0]
precip_dia22, lats, lons = grb.data(lat1=extent[1],lat2=extent[3],lon1=extent[0]+360,lon2=extent[2]+360)

# DIA 23
grib = pygrib.open('/content/MERGE_CPTEC_20250823.grib2')
grb = grib.select(name='Precipitation')[0]
precip_dia23, lats, lons = grb.data(lat1=extent[1],lat2=extent[3],lon1=extent[0]+360,lon2=extent[2]+360)

# DIA 24
grib = pygrib.open('/content/MERGE_CPTEC_20250824.grib2')
grb = grib.select(name='Precipitation')[0]
precip_dia24, lats, lons = grb.data(lat1=extent[1],lat2=extent[3],lon1=extent[0]+360,lon2=extent[2]+360)

#==================================================================================================#
#                      SOMA CHUVA A ENTRE OS DIAS 20 e 24
#==================================================================================================#
# inicializa o array
precip_sum = np.zeros((precip_dia24.shape[0], precip_dia24.shape[1]))

# soma os valores
precip_sum = precip_dia20 + precip_dia21 + precip_dia22 + precip_dia23 + precip_dia24

In [ ]:
precip_dia23

In [ ]:
precip_sum

# **Plota figura do MERGE**

In [ ]:
#==================================================================================================#
#                                       CARREGA SHAPEFILE
#==================================================================================================#
ESTADO = 'RS'
MUNICIPIO = 'Canguçu'
NAME = f'{MUNICIPIO}/{ESTADO}'
shapefile_regiao_cangucu = carrega_shape_municipio(MUNICIPIO)

ESTADO = 'RS'
MUNICIPIO = 'São Gabriel'
NAME = f'{MUNICIPIO}/{ESTADO}'
shapefile_regiao_saogabriel = carrega_shape_municipio(MUNICIPIO)

#==================================================================================================#
#                              DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = uplt.subplots(ncols=3, nrows=2, axheight=6.9, axwidth=6.8, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=True, borders=True, innerborders=True,
          labels=True, latlines=5, lonlines=5,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          abc=False, #abcstyle='(a)', abcsize=5
          small='40px', large='40px',
          linewidth=2,
          suptitle='Acumulado de Precipitação', suptitlecolor='red', suptitlesize=40)

# título de nome do produto
ax[3].text(lonmin+0.1, latmin-2.5, 'MERGE/10km', color='grey', fontsize=30)

#==================================================================================================#
#                                       PALETA DE CORES
#==================================================================================================#
# define as 24 cores da paleta de PRECIPITAÇÃO
colors = ["#b4f0f0", "#96d2fa", "#78b9fa", "#3c95f5", "#1e6deb", "#1463d2", "#0fa00f",
          "#28be28", "#50f050", "#72f06e", "#b3faaa", "#fff9aa", "#ffe978", "#ffc13c",
          "#ffa200", "#ff6200", "#ff3300", "#ff1500", "#c00100", "#a50200", "#870000",
          "#653b32"]

# carrega a paleta de cores através do cmap
cmap = matplotlib.colors.ListedColormap(colors)

# seta a cor para valores acima do valor máximo
cmap.set_over('#000000')

# seta a cor para valores abaixo do valor mínimo
cmap.set_under('#aaaaaa')

#==================================================================================================#
#                                       PLOTA FIGURAS
#==================================================================================================#
# datas das figuras
datas = ['2025-08-20', '2025-08-21', '2025-08-22', '2025-08-23', '2025-08-24','acumulado']

# loop das datas
for i, data in enumerate(datas):

    # define o dado de cada dia
    if data == '2025-08-20': precip_dia = precip_dia20
    if data == '2025-08-21': precip_dia = precip_dia21
    if data == '2025-08-22': precip_dia = precip_dia22
    if data == '2025-08-23': precip_dia = precip_dia23
    if data == '2025-08-24': precip_dia = precip_dia24
    if data == 'acumulado': precip_dia = precip_sum

    # figura
    map1 = ax[i].contourf(lons, lats, precip_dia, cmap=cmap, levels=uplt.arange(0.01, 200, 10), extend='max')

    # título de cada figura
    if i == 0: ax[i].format(title=data, labels=[True, False, False, False])
    if i == 1: ax[i].format(title=data, labels=[False, False, False, False])
    if i == 2: ax[i].format(title=data, labels=[False, False, False, False])
    if i == 3: ax[i].format(title=data, labels=[True, False, True, False])
    if i == 4: ax[i].format(title=data, labels=[False, False, True, False])
    if i == 5: ax[i].format(title='Acumulado Total', labels=[False, False, True, False])

    # valor de chuva
    ax[i].text(lonmax-9.3, latmin+0.5, f'Precip. Máx: {round(precip_dia.max(), 1)} mm ', color='black', fontsize=20)

    # contornos dos ESTADOS
    ax[i].add_geometries(shp_ES, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_RJ, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_SP, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_PR, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_SC, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_RS, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)

    # shapefile de Porto Alegre
    shapefile_regiao_cangucu.plot(edgecolor='red', facecolor='none', linewidth=2, alpha=1, ax=ax[i])
    shapefile_regiao_saogabriel.plot(edgecolor='black', facecolor='none', linewidth=2, alpha=1, ax=ax[i])

# barra de cores
fig.colorbar(map1, loc='b', label='mm', ticks=25, ticklabelsize=30, labelsize=30, space=3.1, length=0.55, width=0.6)

# salva figura
fig.save(f'{dir}/output/merge/merge_2025-08-20a24_RS.jpg', dpi=300)

# exibe a figura na tela
uplt.show()